In [ ]:
pip install seaborn

In [ ]:
pip install statsmodels

In [ ]:
pip install xgboost

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn as skl
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV, MultiTaskLassoCV, MultiTaskElasticNetCV, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor,MultiOutputClassifier
from tqdm import tqdm 
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

In [3]:
from sklearn.ensemble import RandomForestClassifier


In [4]:
from statsmodels.tsa.stattools import acf

In [5]:

from xgboost import XGBClassifier

# I- Training linear models on daily data 

We will try to train models on both daily and weekly data. We may see some different dynamics between daily and weekly data: it could be easier to train a model on weekly data because there is less autocorrelation between datapoints. However in the meanwhile, the dataset on daily datapoints may be bigger so it could also be easier to train models on daily data. 

In [ ]:
datapath = os.path.join('data', 'US', 'us_data.csv')
dus = pd.read_csv(datapath, index_col=0)

In [ ]:
cols_inf = dus.columns[np.isinf(dus.to_numpy()).any(axis=0)]
print(cols_inf)

In [ ]:
dus.hist(figsize=(23, 23), bins=100)
plt.tight_layout()

### A) Creating new features.

Now we need to add the lagged values of yields as features. We have to choose lags and yields to add as features. 

In [ ]:

for USyield in ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3','DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30']:
    series = dus[USyield]
    fig, axes = plt.subplots(1,2, figsize=(8,4))
    plot_acf(series, lags=90,title = f'ACF {USyield}', ax = axes[0])
    plot_pacf(series, lags=90,title = f'PACF {USyield}',ax = axes[1])

    for ax in axes:
        ax.set_xlim(0.5, 90)  # décale le début après 0
        ax.set_ylim(-0.2, 0.2) 

        if USyield in ['DGS5','DGS7','DGS10', 'DGS20', 'DGS30']:
            
            ax.axvline(x=25, color='red', linestyle='--', linewidth=1)
            ax.axvline(x=50, color='red', linestyle='--', linewidth=1)

    
    plt.show()


- We can see that on short term yields, there is much more autocorrelation in the data, up to more than 30 days. Returns in the past few days are highly correlated to returns in the next days. 
- However, on long term yields, there is much less autocorrelation and returns in the past 2 days are only slightly correlated to next day return. Surprisingly we see some persistent autocorrelation between returns at day t and t-25 and t-50. 

For maturities less than 1y, we'll add the following lags:
- t-1,t-2,t-5,t-10,t-15,t-20,t-25,t-30,t-40,t-50

For maturities more than 1y, we will add:
- t-1,t-2,t-10,t-25,t-50

We will probably need to do some PCA to combine features as they will be very correlated.

In [ ]:
for lag in [1,2,5,10,15,20,25,30,40,50]:
    dus[f'DGS1MO_t-{lag}'] = dus['DGS1MO'].shift(lag-1)
    dus[f'DGS3MO_t-{lag}'] = dus['DGS3MO'].shift(lag-1)
    dus[f'DGS6MO_t-{lag}'] = dus['DGS6MO'].shift(lag-1)
    dus[f'DGS1_t-{lag}'] = dus['DGS1'].shift(lag-1)
  

for lag in [1,2,10,15,25,50]:
    dus[f'DGS1_t-{lag}'] = dus['DGS1'].shift(lag-1)
    dus[f'DGS2_t-{lag}'] = dus['DGS2'].shift(lag-1)
    dus[f'DGS3_t-{lag}'] = dus['DGS3'].shift(lag-1)
    dus[f'DGS5_t-{lag}'] = dus['DGS5'].shift(lag-1)
    dus[f'DGS7_t-{lag}'] = dus['DGS7'].shift(lag-1)
    dus[f'DGS10_t-{lag}'] = dus['DGS10'].shift(lag-1)
    dus[f'DGS20_t-{lag}'] = dus['DGS20'].shift(lag-1)
    dus[f'DGS30_t-{lag}'] = dus['DGS30'].shift(lag-1)

In [ ]:
#creating variables to forecast 

dus['Y_1MO'] = dus['DGS1MO'].shift(-1)
dus['Y_3MO'] = dus['DGS3MO'].shift(-1)
dus['Y_6MO'] = dus['DGS6MO'].shift(-1)
dus['Y_1year'] = dus['DGS1'].shift(-1)
dus['Y_2year'] = dus['DGS2'].shift(-1)
dus['Y_3year'] = dus['DGS3'].shift(-1)
dus['Y_5year'] = dus['DGS5'].shift(-1)
dus['Y_7year'] = dus['DGS7'].shift(-1)
dus['Y_10year'] = dus['DGS10'].shift(-1)
dus['Y_20year'] = dus['DGS20'].shift(-1)
dus['Y_30year'] = dus['DGS30'].shift(-1)

In [ ]:
#we can now remove the original yield columns
dus = dus.drop(columns=['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3','DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30'])

We can now look at the heatmap:

In [ ]:
plt.figure(figsize = (25,25))
sns.heatmap(dus.corr(), cmap='seismic', center=0)

Overall the features have a very low correlation with the target, so we'll remove the least correlated ones: 

- the lagged features that have a correlation coefficient < 0.05 in absolute value with all target variables. 
- the other features that have a correlation coefficient < 0.03 in absolute value with all target variables. We do a distinction between lagged features and other features because filtering all features with the 0.05 threshold removes somes features that should have a predictive impact: sp500, gold, VIX for instance. 

Moreover, given the very high correlation between lagged features, we'll apply a PCA in the pipeline on those to limit the number of colinear features.

In [ ]:
Y = dus[['Y_1MO', 'Y_3MO', 'Y_6MO', 'Y_1year', 'Y_2year', 'Y_3year', 'Y_5year', 'Y_7year', 'Y_10year', 'Y_20year', 'Y_30year']]
dus_lagged_features = dus[[col for col in dus.columns if '_t-' in col]]
dus_other =dus.drop(columns=[col for col in dus.columns if '_t-' in col])

corrs = pd.DataFrame({
    target: dus_lagged_features.corrwith(Y[target]) for target in Y.columns
}).abs()  

# repérer les colonnes où la corrélation absolue < 0.05 pour toutes les targets
mask = (corrs < 0.05).all(axis=1)
low_corr_features = corrs.index[mask]

# supprimer ces colonnes
dus_filtered = dus.drop(columns=low_corr_features)



corrs = pd.DataFrame({
    target: dus_other.corrwith(Y[target]) for target in Y.columns
}).abs()  

# repérer les colonnes où la corrélation absolue < 0.05 pour toutes les targets
mask = (corrs < 0.03).all(axis=1)
low_corr_features_2 = corrs.index[mask]

# supprimer ces colonnes
dus_filtered = dus_filtered.drop(columns=low_corr_features_2)

print(f"{len(low_corr_features) + len(low_corr_features_2)} features were deleted")

In [ ]:
print(low_corr_features)
print(low_corr_features_2)

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(dus_filtered.corr(), cmap='seismic', center=0)

### B) PCA 

In [ ]:
dus_filtered = dus_filtered.dropna()
X = dus_filtered[['USGOOD', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP', 'USWTRADE',
       'USFIRE', 'PERMIT', 'UMCSENT', 'M2SL', 'M2REAL', 'TOTRESNS', 'CPIAUCSL',
       'CPIAPPSL', 'CPITRNSL', 'CUSR0000SAC', 'CPIULFSL', 'CUUR0000SA0L2',
       'PCEPI', 'DNDGRG3M086SBEA', 'MTSDS133FMS', 'GFDEGDQ188S',
       'IRLTLT01DEM156N', 'IRLTLT01JPM156N', 'IRLTLT01GBM156N',
       'IRLTLT01CAM156N', 'IRLTLT01AUM156N', 'IRLTLT01FRM156N', 'NASDAQCOM',
       'AAA', 'BAA', 'DEXCAUS', 'DEXUSAL', 'NFCI', 'FEDFUNDS', 'BOGMBASE',
       'WSHOSHO', 'T5YIE', 'T10YIE', 'log return gold', 'log return sp500',
       'DGS1MO_t-1', 'DGS3MO_t-1', 'DGS6MO_t-1', 'DGS1_t-1', 'DGS1MO_t-2',
       'DGS3MO_t-2', 'DGS6MO_t-2', 'DGS1MO_t-5', 'DGS3MO_t-5', 'DGS6MO_t-5',
       'DGS3MO_t-10', 'DGS6MO_t-10', 'DGS1_t-10', 'DGS1MO_t-15', 'DGS3MO_t-15',
       'DGS6MO_t-15', 'DGS1MO_t-20', 'DGS3MO_t-20', 'DGS6MO_t-20', 'DGS1_t-20',
       'DGS3MO_t-30', 'DGS6MO_t-40', 'DGS1MO_t-50', 'DGS2_t-1', 'DGS3_t-1',
       'DGS10_t-1', 'DGS20_t-1', 'DGS30_t-1', 'DGS7_t-50', 'DGS10_t-50',
       'DGS20_t-50', 'DGS30_t-50']]

Y = dus_filtered[['Y_1MO', 'Y_3MO', 'Y_6MO', 'Y_1year', 'Y_2year', 'Y_3year', 'Y_5year', 'Y_7year', 'Y_10year', 'Y_20year', 'Y_30year']]


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

print("Number of components kept:", pca.n_components_)
print("cumulative explained variance :", pca.explained_variance_ratio_.cumsum())
print("Composantes principales (coefficients sur les features originales) :")
print(pca.components_)

print("Exemple des nouvelles features transformées :")
print(X_pca[:5])

In [ ]:
# calculer la contribution absolue sur PC1 et PC2
pc1, pc2 = np.abs(pca.components_[:2])
importance = pc1 + pc2

# garder les n features les plus importantes
n = 30
top_idx = np.argsort(importance)[-n:]
top_labels = X.columns[top_idx]
top_components = pca.components_[:2, top_idx]

plt.figure(figsize=(12,12))
circle = plt.Circle((0,0), 1, color='gray', fill=False)
plt.gca().add_artist(circle)

for i, (x, y) in enumerate(zip(top_components[0,:], top_components[1,:])):
    plt.arrow(0, 0, x, y, color='r', alpha=0.6, head_width=0.02)
    plt.text(x*1.15, y*1.15, top_labels[i], color='b', ha='center', va='center', fontsize=9)

plt.xlim(-1.1, 1.1)
plt.ylim(-1.1, 1.1)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Correlation circle (top features)")
plt.grid()
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.show()


We can see that features are overall very correlated, so we'll train some models with PCA and some without.

### C) Training a ridge model

We do a walk forward cross validation:
- we train our model on 4 years of data (= approximately 1000 data points)
- we do prediction for the next month (21 days)
- wa add a PCA to the pipeline to deal with correlated features. We'll also train a model without PCA to see how it changes the performance.

In [ ]:


window_train = 252 *4
window_pred = 21          
alphas = np.logspace(-3, 3, 20)


tscv = TimeSeriesSplit(n_splits=4)

pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('pca',PCA(n_components=0.95)),   
    ('ridge', MultiOutputRegressor(RidgeCV(fit_intercept=False,alphas=alphas, cv=tscv)))
])

pipe_nopca = Pipeline([
    ('scaler',StandardScaler()),   
    ('ridge', MultiOutputRegressor(RidgeCV(fit_intercept=False,alphas=alphas, cv=tscv)))
])

preds = []
dates_pred = []
r2_is_list = []
r2_os_list=[]
hit_rate_list = []

preds_nopca = []
dates_pred_nopca = []
r2_is_list_nopca = []
r2_os_list_nopca=[]
hit_rate_list_nopca = []

for start in tqdm(range(0, len(X) - window_train - window_pred + 1, window_pred)):
  
    end_train = start + window_train
    end_pred = end_train + window_pred

    X_train = X.iloc[start:end_train]
    Y_train = Y.iloc[start:end_train]

    X_test = X.iloc[end_train:end_pred]
    Y_test = Y.iloc[end_train:end_pred]


    # model with PCA 
    pipe.fit(X_train, Y_train)

    Y_pred = pipe.predict(X_test)

    preds.append(Y_pred)
    dates_pred.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred), axis=0)
    r2_is_list.append(r2_is)
    r2_os_list.append(r2_oos)
    hit_rate_list.append(hit_rate)


    # model without PCA 
    pipe_nopca.fit(X_train, Y_train)

    Y_pred_nopca = pipe_nopca.predict(X_test)

    preds_nopca.append(Y_pred)
    dates_pred_nopca.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred_nopca, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe_nopca.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred_nopca, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred_nopca), axis=0)
    r2_is_list_nopca.append(r2_is)
    r2_os_list_nopca.append(r2_oos)
    hit_rate_list_nopca.append(hit_rate)





In [ ]:
r2_is_df = pd.DataFrame(r2_is_list) 
r2_is_df_nopca = pd.DataFrame(r2_is_list_nopca) 
r2_is_df.columns = Y.columns
r2_is_df_nopca.columns = Y.columns
r2_is_df.index = [date[0] for date in dates_pred]
r2_is_df_nopca.index = [date[0] for date in dates_pred]


r2_os_df = pd.DataFrame(r2_os_list)  
r2_os_df.columns = Y.columns
r2_os_df.index = [date[0] for date in dates_pred]
r2_os_df_nopca = pd.DataFrame(r2_os_list_nopca) 
r2_os_df_nopca.columns = Y.columns
r2_os_df_nopca.index = [date[0] for date in dates_pred]


hr = pd.DataFrame(hit_rate_list) 
hr.columns = Y.columns
hr.index = [date[0] for date in dates_pred]
hr_nopca = pd.DataFrame(hit_rate_list_nopca) 
hr_nopca.columns = Y.columns
hr_nopca.index = [date[0] for date in dates_pred]


plt.close('all')  # ferme toutes les figures existantes


fig, ax = plt.subplots(3,2,figsize=(28,28))

r2_is_df.plot(ax = ax[0,0],title = 'Evolution of in sample R2 per model with PCA, per sample')
ax[0,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_is_df_nopca.plot(ax = ax[0,1],title = 'Evolution of in sample R2 per model without PCA, per sample')
ax[0,1].grid(True, axis='y', linestyle='--', linewidth=0.5)


r2_os_df.plot(ax = ax[1,0], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per model with PCA, per sample')
ax[1,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_os_df_nopca.plot(ax = ax[1,1], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per model without PCA, per sample')
ax[1,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

hr.plot(ax = ax[2,0], title = 'Evolution of out-sample hit rate per model with PCA, per sample')
ax[2,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
hr_nopca.plot(ax = ax[2,1], title = 'Evolution of out-sample hit rate per model without PCA, per sample')
ax[2,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

- When looking at the in sample R2 of our models, we see that the R2 is overall low, but that it is better on short term yields, ie from 1 month to 1 year, and significantly lower for long-term yields. 
- we see significant variations in R2 in 2011 and 2020, probably because of outliers. 

- The out of sample R2 is close to zero or even negative so there is no predictive power in our model.

- adding a PCA in the pipeline do not change anything to the in sample R2. However, the hit rate is slightly less variable when adding the PCA, suggesting a bit less overfitting (although the out of sample results are as bad).
- The hit rate seems to be close to 0.5 on average for all models and all samples. So it appears the model does not do anything better than predicting at random - it completely overfits the data.



## Training Lasso

In [ ]:
np.logspace(-5, 2, 20)

In [ ]:
window_train = 252 *4
window_pred = 21          
alphas = np.logspace(-5, 2, 20)


tscv = TimeSeriesSplit(n_splits=4)

pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('pca',PCA(n_components=0.95)),   
    ('lasso', MultiTaskLassoCV(fit_intercept=False,alphas=alphas, cv=tscv))
])

pipe_nopca = Pipeline([
    ('scaler',StandardScaler()),   
    ('lasso', MultiTaskLassoCV(fit_intercept=False,alphas=alphas, cv=tscv))
])

preds = []
dates_pred = []
r2_is_list = []
r2_os_list=[]
hit_rate_list = []
alpha,selected_features = [],[]

preds_nopca = []
dates_pred_nopca = []
r2_is_list_nopca = []
r2_os_list_nopca=[]
hit_rate_list_nopca = []
alpha_nopca,selected_features_nopca = [],[]

for start in tqdm(range(0, len(X) - window_train - window_pred + 1, window_pred)):
  
    end_train = start + window_train
    end_pred = end_train + window_pred

    X_train = X.iloc[start:end_train]
    Y_train = Y.iloc[start:end_train]

    X_test = X.iloc[end_train:end_pred]
    Y_test = Y.iloc[end_train:end_pred]


    # model with PCA 
    pipe.fit(X_train, Y_train)

    Y_pred = pipe.predict(X_test)

    preds.append(Y_pred)
    dates_pred.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred), axis=0)
    r2_is_list.append(r2_is)
    r2_os_list.append(r2_oos)
    hit_rate_list.append(hit_rate)
    alpha.append(pipe.named_steps['lasso'].alpha_)
    selected_features.append(np.sum(np.any(pipe.named_steps['lasso'].coef_!=0,axis=0)))


    # model without PCA 
    pipe_nopca.fit(X_train, Y_train)

    Y_pred_nopca = pipe_nopca.predict(X_test)

    preds_nopca.append(Y_pred)
    dates_pred_nopca.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred_nopca, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe_nopca.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred_nopca, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred_nopca), axis=0)
    r2_is_list_nopca.append(r2_is)
    r2_os_list_nopca.append(r2_oos)
    hit_rate_list_nopca.append(hit_rate)
    alpha_nopca.append(pipe_nopca.named_steps['lasso'].alpha_)
    selected_features_nopca.append(np.sum(np.any(pipe_nopca.named_steps['lasso'].coef_!=0,axis=0)))






In [ ]:
r2_is_df = pd.DataFrame(r2_is_list) 
r2_is_df_nopca = pd.DataFrame(r2_is_list_nopca) 
r2_is_df.columns = Y.columns
r2_is_df_nopca.columns = Y.columns
r2_is_df.index = [date[0] for date in dates_pred]
r2_is_df_nopca.index = [date[0] for date in dates_pred]


r2_os_df = pd.DataFrame(r2_os_list)  
r2_os_df.columns = Y.columns
r2_os_df.index = [date[0] for date in dates_pred]
r2_os_df_nopca = pd.DataFrame(r2_os_list_nopca) 
r2_os_df_nopca.columns = Y.columns
r2_os_df_nopca.index = [date[0] for date in dates_pred]


hr = pd.DataFrame(hit_rate_list) 
hr.columns = Y.columns
hr.index = [date[0] for date in dates_pred]
hr_nopca = pd.DataFrame(hit_rate_list_nopca) 
hr_nopca.columns = Y.columns
hr_nopca.index = [date[0] for date in dates_pred]

fs = pd.DataFrame(selected_features)
fs.index = [date[0] for date in dates_pred]

fs_nopca = pd.DataFrame(selected_features_nopca)
fs_nopca.index = [date[0] for date in dates_pred]


alphadf = pd.DataFrame(alpha)
alphadf.index = [date[0] for date in dates_pred]

alphadf_nopca = pd.DataFrame(alpha_nopca)
alphadf_nopca.index = [date[0] for date in dates_pred]




fig, ax = plt.subplots(5,2,figsize=(28,28))

r2_is_df.plot(ax = ax[0,0],title = 'Evolution of in sample R2 per maturity with PCA, per sample')
ax[0,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_is_df_nopca.plot(ax = ax[0,1],title = 'Evolution of in sample R2 per maturity without PCA, per sample')
ax[0,1].grid(True, axis='y', linestyle='--', linewidth=0.5)


r2_os_df.plot(ax = ax[1,0], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per maturity with PCA, per sample')
ax[1,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_os_df_nopca.plot(ax = ax[1,1], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per maturity without PCA, per sample')
ax[1,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

hr.plot(ax = ax[2,0], title = 'Evolution of out-sample hit rate per maturity with PCA, per sample')
ax[2,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
hr_nopca.plot(ax = ax[2,1], title = 'Evolution of out-sample hit rate per maturity without PCA, per sample')
ax[2,1].grid(True, axis='y', linestyle='--', linewidth=0.5)


fs.plot(ax = ax[3,0], title = 'Evolution of number of selected features with PCA, per sample')
ax[3,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
fs_nopca.plot(ax = ax[3,1], title = 'Evolution of number of selected features without PCA, per sample')
ax[3,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

alphadf.plot(ax = ax[4,0], title = 'Evolution of alpha selected by cross validation - model with PCA, per sample')
ax[4,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
alphadf_nopca.plot(ax = ax[4,1], title = 'Evolution of alpha selected by cross validation - model without PCA, per sample')
ax[4,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

Lasso performance is even worse than that of ridge because there are some periods for which the model selects no features. 

## Training Elastic net 

In [ ]:
window_train = 252 *10
window_pred = 21          
alphas = np.logspace(-3, 2, 20)
l1_ratios = [0.25,0.5,0.75]


tscv = TimeSeriesSplit(n_splits=4)

pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('pca',PCA(n_components=0.95)),   
    ('elasticnet', MultiTaskElasticNetCV(alphas=alphas, cv=tscv, l1_ratio=0.5, fit_intercept=False,max_iter=5000))
])

pipe_nopca = Pipeline([
    ('scaler',StandardScaler()),   
    ('elasticnet', MultiTaskElasticNetCV(alphas=alphas, cv=tscv, l1_ratio=l1_ratios, fit_intercept=False,max_iter=5000))
])

preds = []
dates_pred = []
r2_is_list = []
r2_os_list=[]
hit_rate_list = []
alpha,selected_features,l1 = [],[],[]

preds_nopca = []
dates_pred_nopca = []
r2_is_list_nopca = []
r2_os_list_nopca=[]
hit_rate_list_nopca = []
alpha_nopca,selected_features_nopca,l1_nopca = [],[],[]

for start in tqdm(range(0, len(X) - window_train - window_pred + 1, window_pred)):
  
    end_train = start + window_train
    end_pred = end_train + window_pred

    X_train = X.iloc[start:end_train]
    Y_train = Y.iloc[start:end_train]

    X_test = X.iloc[end_train:end_pred]
    Y_test = Y.iloc[end_train:end_pred]


    # model with PCA 
    pipe.fit(X_train, Y_train)

    Y_pred = pipe.predict(X_test)

    preds.append(Y_pred)
    dates_pred.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred), axis=0)
    r2_is_list.append(r2_is)
    r2_os_list.append(r2_oos)
    hit_rate_list.append(hit_rate)
    alpha.append(pipe.named_steps['elasticnet'].alpha_)
    selected_features.append(np.sum(np.any(pipe.named_steps['elasticnet'].coef_!=0,axis=0)))
    l1.append(pipe.named_steps['elasticnet'].l1_ratio_)


    # model without PCA 
    pipe_nopca.fit(X_train, Y_train)

    Y_pred_nopca = pipe_nopca.predict(X_test)

    preds_nopca.append(Y_pred)
    dates_pred_nopca.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred_nopca, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe_nopca.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred_nopca, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred_nopca), axis=0)
    r2_is_list_nopca.append(r2_is)
    r2_os_list_nopca.append(r2_oos)
    hit_rate_list_nopca.append(hit_rate)
    alpha_nopca.append(pipe_nopca.named_steps['elasticnet'].alpha_)
    selected_features_nopca.append(np.sum(np.any(pipe_nopca.named_steps['elasticnet'].coef_!=0,axis=0)))
    l1_nopca.append(pipe_nopca.named_steps['elasticnet'].l1_ratio_)






In [ ]:
r2_is_df = pd.DataFrame(r2_is_list) 
r2_is_df_nopca = pd.DataFrame(r2_is_list_nopca) 
r2_is_df.columns = Y.columns
r2_is_df_nopca.columns = Y.columns
r2_is_df.index = [date[0] for date in dates_pred]
r2_is_df_nopca.index = [date[0] for date in dates_pred]


r2_os_df = pd.DataFrame(r2_os_list)  
r2_os_df.columns = Y.columns
r2_os_df.index = [date[0] for date in dates_pred]
r2_os_df_nopca = pd.DataFrame(r2_os_list_nopca) 
r2_os_df_nopca.columns = Y.columns
r2_os_df_nopca.index = [date[0] for date in dates_pred]


hr = pd.DataFrame(hit_rate_list) 
hr.columns = Y.columns
hr.index = [date[0] for date in dates_pred]
hr_nopca = pd.DataFrame(hit_rate_list_nopca) 
hr_nopca.columns = Y.columns
hr_nopca.index = [date[0] for date in dates_pred]

fs = pd.DataFrame(selected_features)
fs.index = [date[0] for date in dates_pred]

fs_nopca = pd.DataFrame(selected_features_nopca)
fs_nopca.index = [date[0] for date in dates_pred]


alphadf = pd.DataFrame(alpha)
alphadf.index = [date[0] for date in dates_pred]

alphadf_nopca = pd.DataFrame(alpha_nopca)
alphadf_nopca.index = [date[0] for date in dates_pred]


l1df = pd.DataFrame(l1)
l1df.index = [date[0] for date in dates_pred]

l1df_nopca = pd.DataFrame(l1_nopca)
l1df_nopca.index = [date[0] for date in dates_pred]







fig, ax = plt.subplots(6,2,figsize=(28,28))

r2_is_df.plot(ax = ax[0,0],title = 'Evolution of in sample R2 per maturity with PCA, per sample')
ax[0,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_is_df_nopca.plot(ax = ax[0,1],title = 'Evolution of in sample R2 per maturity without PCA, per sample')
ax[0,1].grid(True, axis='y', linestyle='--', linewidth=0.5)


r2_os_df.plot(ax = ax[1,0], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per maturity with PCA, per sample')
ax[1,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_os_df_nopca.plot(ax = ax[1,1], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per maturity without PCA, per sample')
ax[1,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

hr.plot(ax = ax[2,0], title = 'Evolution of out-sample hit rate per maturity with PCA, per sample')
ax[2,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
hr_nopca.plot(ax = ax[2,1], title = 'Evolution of out-sample hit rate per maturity without PCA, per sample')
ax[2,1].grid(True, axis='y', linestyle='--', linewidth=0.5)


fs.plot(ax = ax[3,0], title = 'Evolution of number of selected features with PCA, per sample')
ax[3,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
fs_nopca.plot(ax = ax[3,1], title = 'Evolution of number of selected features without PCA, per sample')
ax[3,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

alphadf.plot(ax = ax[4,0], title = 'Evolution of alpha selected by cross validation - model with PCA, per sample')
ax[4,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
alphadf_nopca.plot(ax = ax[4,1], title = 'Evolution of alpha selected by cross validation - model without PCA, per sample')
ax[4,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

l1df.plot(ax = ax[5,0], title = 'Evolution of L1 ratio selected by cross validation - model with PCA, per sample')
ax[5,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
l1df_nopca.plot(ax = ax[5,1], title = 'Evolution of L1 ratio selected by cross validation - model without PCA, per sample')
ax[5,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

Overall all these linear models work very poorly, which is not surprising given the low correlation between features and past yields. We can now try to work on weekly data, which may be a bit less noisy: hopefully our model will be able to better identify some trends and patterns. 

# II) Training linear models on weekly data 

In [ ]:
datapath = os.path.join('data', 'US', 'us_data_weekly.csv')
dus = pd.read_csv(datapath, index_col=0)

In [ ]:
dus = dus[dus.index>'2003-01-03']

### A) Creating new features

In [ ]:

for USyield in ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3','DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30']:
    series = dus[USyield]
    fig, axes = plt.subplots(1,2, figsize=(14,6))
    plot_acf(series, lags=50,title = f'ACF {USyield}', ax = axes[0])
    plot_pacf(series, lags=50,title = f'PACF {USyield}',ax = axes[1])
    

    for ax in axes:
        ax.set_xlim(0.5, 50)  # décale le début après 0
        ax.set_ylim(-0.2, 0.2) 

    
    plt.show()


- We can see that on short term yields, there is much more autocorrelation in the data, up to 40 lags. Returns in the past few weeks (and so on the past year) are highly correlated to returns in the next week. 
- However, on long term yields, there is much less autocorrelation and returns in the past 2 weeks are only very slightly correlated to next day return. Surprisingly we see some persistent autocorrelation between returns at day t and t-25 and t-50. 

For maturities less than 1y, we'll add the following lags:
- t-1,t-2,t-5,t-10,t-15,t-20,t-25,t-30,t-40

For maturities more than 1y, we will add:
- t-1,t-2,t-10

We will probably need to do some PCA to combine features as they will be very correlated.

In [ ]:
for lag in [1,2,5,10,15,20,25,30,40,50]:
    dus[f'DGS1MO_t-{lag}'] = dus['DGS1MO'].shift(lag-1)
    dus[f'DGS3MO_t-{lag}'] = dus['DGS3MO'].shift(lag-1)
    dus[f'DGS6MO_t-{lag}'] = dus['DGS6MO'].shift(lag-1)
    dus[f'DGS1_t-{lag}'] = dus['DGS1'].shift(lag-1)
  

for lag in [1,2,10]:
    dus[f'DGS1_t-{lag}'] = dus['DGS1'].shift(lag-1)
    dus[f'DGS2_t-{lag}'] = dus['DGS2'].shift(lag-1)
    dus[f'DGS3_t-{lag}'] = dus['DGS3'].shift(lag-1)
    dus[f'DGS5_t-{lag}'] = dus['DGS5'].shift(lag-1)
    dus[f'DGS7_t-{lag}'] = dus['DGS7'].shift(lag-1)
    dus[f'DGS10_t-{lag}'] = dus['DGS10'].shift(lag-1)
    dus[f'DGS20_t-{lag}'] = dus['DGS20'].shift(lag-1)
    dus[f'DGS30_t-{lag}'] = dus['DGS30'].shift(lag-1)

In [ ]:
#creating variables to forecast 

dus['Y_1MO'] = dus['DGS1MO'].shift(-1)
dus['Y_3MO'] = dus['DGS3MO'].shift(-1)
dus['Y_6MO'] = dus['DGS6MO'].shift(-1)
dus['Y_1year'] = dus['DGS1'].shift(-1)
dus['Y_2year'] = dus['DGS2'].shift(-1)
dus['Y_3year'] = dus['DGS3'].shift(-1)
dus['Y_5year'] = dus['DGS5'].shift(-1)
dus['Y_7year'] = dus['DGS7'].shift(-1)
dus['Y_10year'] = dus['DGS10'].shift(-1)
dus['Y_20year'] = dus['DGS20'].shift(-1)
dus['Y_30year'] = dus['DGS30'].shift(-1)

We'll also add statistical features like the mean, variance, autocorrelation, quantiles of the time series to forecast.

In [ ]:
def add_ts_features(df, cols, max_lag=30, windows=[20, 60]):
    features = pd.DataFrame(index=df.index)
    
    for col in cols:
        y = df[col]

  
        # --- Statistiques glissantes ---
        for w in windows:
            features[f'{col}_mean_{w}'] = y.rolling(w).mean()
            features[f'{col}_std_{w}'] = y.rolling(w).std()
            features[f'{col}_q25_{w}'] = y.rolling(w).quantile(0.25)
            features[f'{col}_q75_{w}'] = y.rolling(w).quantile(0.75)
            features[f'{col}_q05_{w}'] = y.rolling(w).quantile(0.1)
            features[f'{col}_q90_{w}'] = y.rolling(w).quantile(0.9)
            features[f'{col}_range_{w}'] = y.rolling(w).max() - y.rolling(w).min() 
        
       
        # --- Autocorrélations locales ---
        for lag in range(1, max_lag + 1):
            features[f'{col}_autocorr_{lag}'] = (
                y.rolling(window=max(windows)).apply(lambda x: x.autocorr(lag=lag), raw=False)
            )
        
    return features

# Exemple d’usage :
cols = ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3',
        'DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30']

dus_features = add_ts_features(dus, cols)


In [ ]:
dus_features.dropna()

In [ ]:
dus = dus.merge(dus_features, how = 'inner', left_index=True, right_index=True)

In [ ]:
dus = dus.dropna()
dus

In [ ]:
#we can now remove the original yield columns
dus = dus.drop(columns=['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3','DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30'])

Many features aren't that much correlated to the target, so we'll remove all features with correlation lower than 0.1 to the target.

In [ ]:
Y = dus[['Y_1MO', 'Y_3MO', 'Y_6MO', 'Y_1year', 'Y_2year', 'Y_3year', 'Y_5year', 'Y_7year', 'Y_10year', 'Y_20year', 'Y_30year']]

corrs = pd.DataFrame({
    target: dus.corrwith(Y[target]) for target in Y.columns
}).abs()  

# repérer les colonnes où la corrélation absolue < 0.05 pour toutes les targets
mask = (corrs < 0.1).all(axis=1)
low_corr_features = corrs.index[mask]

# supprimer ces colonnes
dus_filtered = dus.drop(columns=low_corr_features)

print(f"{len(low_corr_features)} features were deleted")

In [ ]:
print(low_corr_features)

In [ ]:
plt.figure(figsize = (25,25))
sns.heatmap(dus_filtered.corr(), cmap='seismic', center=0)

In [ ]:
print(dus_filtered.shape)

### B) Training a ridge model 

We can now try to train a ridge.

In [ ]:
Y = dus_filtered[['Y_1MO', 'Y_3MO', 'Y_6MO', 'Y_1year', 'Y_2year', 'Y_3year', 'Y_5year', 'Y_7year', 'Y_10year', 'Y_20year', 'Y_30year']]

X = dus_filtered.drop(columns = ['Y_1MO', 'Y_3MO', 'Y_6MO', 'Y_1year', 'Y_2year', 'Y_3year', 'Y_5year', 'Y_7year', 'Y_10year', 'Y_20year', 'Y_30year'])

In [ ]:
window_train = 52*5 #we train our model on 5 years of data and test it on the next month 
window_pred = 4          
alphas = np.logspace(-1, 3, 20)


tscv = TimeSeriesSplit(n_splits=4)

pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('pca',PCA(n_components=0.95)),   
    ('ridge', MultiOutputRegressor(RidgeCV(fit_intercept=False,alphas=alphas, cv=tscv)))
])

pipe_nopca = Pipeline([
    ('scaler',StandardScaler()),   
    ('ridge', MultiOutputRegressor(RidgeCV(fit_intercept=False,alphas=alphas, cv=tscv)))
])

preds = []
dates_pred = []
r2_is_list = []
r2_os_list=[]
hit_rate_list = []

preds_nopca = []
dates_pred_nopca = []
r2_is_list_nopca = []
r2_os_list_nopca=[]
hit_rate_list_nopca = []

for start in tqdm(range(0, len(X) - window_train - window_pred + 1, window_pred)):
  
    end_train = start + window_train
    end_pred = end_train + window_pred

    X_train = X.iloc[start:end_train]
    Y_train = Y.iloc[start:end_train]

    X_test = X.iloc[end_train:end_pred]
    Y_test = Y.iloc[end_train:end_pred]


    # model with PCA 
    pipe.fit(X_train, Y_train)

    Y_pred = pipe.predict(X_test)

    preds.append(Y_pred)
    dates_pred.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred), axis=0)
    r2_is_list.append(r2_is)
    r2_os_list.append(r2_oos)
    hit_rate_list.append(hit_rate)


    # model without PCA 
    pipe_nopca.fit(X_train, Y_train)

    Y_pred_nopca = pipe_nopca.predict(X_test)

    preds_nopca.append(Y_pred)
    dates_pred_nopca.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred_nopca, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe_nopca.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred_nopca, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred_nopca), axis=0)
    r2_is_list_nopca.append(r2_is)
    r2_os_list_nopca.append(r2_oos)
    hit_rate_list_nopca.append(hit_rate)





In [ ]:
r2_is_df = pd.DataFrame(r2_is_list) 
r2_is_df_nopca = pd.DataFrame(r2_is_list_nopca) 
r2_is_df.columns = Y.columns
r2_is_df_nopca.columns = Y.columns
r2_is_df.index = [date[0] for date in dates_pred]
r2_is_df_nopca.index = [date[0] for date in dates_pred]


r2_os_df = pd.DataFrame(r2_os_list)  
r2_os_df.columns = Y.columns
r2_os_df.index = [date[0] for date in dates_pred]
r2_os_df_nopca = pd.DataFrame(r2_os_list_nopca) 
r2_os_df_nopca.columns = Y.columns
r2_os_df_nopca.index = [date[0] for date in dates_pred]


hr = pd.DataFrame(hit_rate_list) 
hr.columns = Y.columns
hr.index = [date[0] for date in dates_pred]
hr_nopca = pd.DataFrame(hit_rate_list_nopca) 
hr_nopca.columns = Y.columns
hr_nopca.index = [date[0] for date in dates_pred]


plt.close('all')  # ferme toutes les figures existantes


fig, ax = plt.subplots(3,2,figsize=(28,28))

r2_is_df.plot(ax = ax[0,0],title = 'Evolution of in sample R2 per model with PCA, per sample')
ax[0,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_is_df_nopca.plot(ax = ax[0,1],title = 'Evolution of in sample R2 per model without PCA, per sample')
ax[0,1].grid(True, axis='y', linestyle='--', linewidth=0.5)


r2_os_df.plot(ax = ax[1,0], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per model with PCA, per sample')
ax[1,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_os_df_nopca.plot(ax = ax[1,1], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per model without PCA, per sample')
ax[1,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

hr.plot(ax = ax[2,0], title = 'Evolution of out-sample hit rate per model with PCA, per sample')
ax[2,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
hr_nopca.plot(ax = ax[2,1], title = 'Evolution of out-sample hit rate per model without PCA, per sample')
ax[2,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

Results are slightly better than when working on daily data, but we still only learn noise and completely overfit the data.

### B) Training an elastic net model

In [ ]:
window_train = 52*5
window_pred = 4         
alphas = np.logspace(-2, 2, 20)
l1_ratios = [0.25,0.5,0.75]


tscv = TimeSeriesSplit(n_splits=4)

pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('pca',PCA(n_components=0.95)),   
    ('elasticnet', MultiTaskElasticNetCV(alphas=alphas, cv=tscv, l1_ratio=0.5, fit_intercept=False,max_iter=5000))
])

pipe_nopca = Pipeline([
    ('scaler',StandardScaler()),   
    ('elasticnet', MultiTaskElasticNetCV(alphas=alphas, cv=tscv, l1_ratio=l1_ratios, fit_intercept=False,max_iter=5000))
])

preds = []
dates_pred = []
r2_is_list = []
r2_os_list=[]
hit_rate_list = []
alpha,selected_features,l1 = [],[],[]

preds_nopca = []
dates_pred_nopca = []
r2_is_list_nopca = []
r2_os_list_nopca=[]
hit_rate_list_nopca = []
alpha_nopca,selected_features_nopca,l1_nopca = [],[],[]

for start in tqdm(range(0, len(X) - window_train - window_pred + 1, window_pred)):
  
    end_train = start + window_train
    end_pred = end_train + window_pred

    X_train = X.iloc[start:end_train]
    Y_train = Y.iloc[start:end_train]

    X_test = X.iloc[end_train:end_pred]
    Y_test = Y.iloc[end_train:end_pred]


    # model with PCA 
    pipe.fit(X_train, Y_train)

    Y_pred = pipe.predict(X_test)

    preds.append(Y_pred)
    dates_pred.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred), axis=0)
    r2_is_list.append(r2_is)
    r2_os_list.append(r2_oos)
    hit_rate_list.append(hit_rate)
    alpha.append(pipe.named_steps['elasticnet'].alpha_)
    selected_features.append(np.sum(np.any(pipe.named_steps['elasticnet'].coef_!=0,axis=0)))
    l1.append(pipe.named_steps['elasticnet'].l1_ratio_)


    # model without PCA 
    pipe_nopca.fit(X_train, Y_train)

    Y_pred_nopca = pipe_nopca.predict(X_test)

    preds_nopca.append(Y_pred)
    dates_pred_nopca.append(X.index[end_train:end_pred])

    mse = mean_squared_error(Y_test, Y_pred_nopca, multioutput='raw_values')
    r2_is = r2_score(Y_train, pipe_nopca.predict(X_train), multioutput='raw_values')
    r2_oos = r2_score(Y_test, Y_pred_nopca, multioutput='raw_values')
    hit_rate = np.mean(np.sign(Y_test.values) == np.sign(Y_pred_nopca), axis=0)
    r2_is_list_nopca.append(r2_is)
    r2_os_list_nopca.append(r2_oos)
    hit_rate_list_nopca.append(hit_rate)
    alpha_nopca.append(pipe_nopca.named_steps['elasticnet'].alpha_)
    selected_features_nopca.append(np.sum(np.any(pipe_nopca.named_steps['elasticnet'].coef_!=0,axis=0)))
    l1_nopca.append(pipe_nopca.named_steps['elasticnet'].l1_ratio_)






In [ ]:
r2_is_df = pd.DataFrame(r2_is_list) 
r2_is_df_nopca = pd.DataFrame(r2_is_list_nopca) 
r2_is_df.columns = Y.columns
r2_is_df_nopca.columns = Y.columns
r2_is_df.index = [date[0] for date in dates_pred]
r2_is_df_nopca.index = [date[0] for date in dates_pred]


r2_os_df = pd.DataFrame(r2_os_list)  
r2_os_df.columns = Y.columns
r2_os_df.index = [date[0] for date in dates_pred]
r2_os_df_nopca = pd.DataFrame(r2_os_list_nopca) 
r2_os_df_nopca.columns = Y.columns
r2_os_df_nopca.index = [date[0] for date in dates_pred]


hr = pd.DataFrame(hit_rate_list) 
hr.columns = Y.columns
hr.index = [date[0] for date in dates_pred]
hr_nopca = pd.DataFrame(hit_rate_list_nopca) 
hr_nopca.columns = Y.columns
hr_nopca.index = [date[0] for date in dates_pred]

fs = pd.DataFrame(selected_features)
fs.index = [date[0] for date in dates_pred]

fs_nopca = pd.DataFrame(selected_features_nopca)
fs_nopca.index = [date[0] for date in dates_pred]


alphadf = pd.DataFrame(alpha)
alphadf.index = [date[0] for date in dates_pred]

alphadf_nopca = pd.DataFrame(alpha_nopca)
alphadf_nopca.index = [date[0] for date in dates_pred]


l1df = pd.DataFrame(l1)
l1df.index = [date[0] for date in dates_pred]

l1df_nopca = pd.DataFrame(l1_nopca)
l1df_nopca.index = [date[0] for date in dates_pred]







fig, ax = plt.subplots(6,2,figsize=(28,28))

r2_is_df.plot(ax = ax[0,0],title = 'Evolution of in sample R2 per maturity with PCA, per sample')
ax[0,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_is_df_nopca.plot(ax = ax[0,1],title = 'Evolution of in sample R2 per maturity without PCA, per sample')
ax[0,1].grid(True, axis='y', linestyle='--', linewidth=0.5)


r2_os_df.plot(ax = ax[1,0], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per maturity with PCA, per sample')
ax[1,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
r2_os_df_nopca.plot(ax = ax[1,1], ylim =(-0.3,0.3), title = 'Evolution of out-sample R2 per maturity without PCA, per sample')
ax[1,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

hr.plot(ax = ax[2,0], title = 'Evolution of out-sample hit rate per maturity with PCA, per sample')
ax[2,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
hr_nopca.plot(ax = ax[2,1], title = 'Evolution of out-sample hit rate per maturity without PCA, per sample')
ax[2,1].grid(True, axis='y', linestyle='--', linewidth=0.5)


fs.plot(ax = ax[3,0], title = 'Evolution of number of selected features with PCA, per sample')
ax[3,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
fs_nopca.plot(ax = ax[3,1], title = 'Evolution of number of selected features without PCA, per sample')
ax[3,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

alphadf.plot(ax = ax[4,0], title = 'Evolution of alpha selected by cross validation - model with PCA, per sample')
ax[4,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
alphadf_nopca.plot(ax = ax[4,1], title = 'Evolution of alpha selected by cross validation - model without PCA, per sample')
ax[4,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

l1df.plot(ax = ax[5,0], title = 'Evolution of L1 ratio selected by cross validation - model with PCA, per sample')
ax[5,0].grid(True, axis='y', linestyle='--', linewidth=0.5)
l1df_nopca.plot(ax = ax[5,1], title = 'Evolution of L1 ratio selected by cross validation - model without PCA, per sample')
ax[5,1].grid(True, axis='y', linestyle='--', linewidth=0.5)

The results are also disappointing. Overall this is not surprising, we've seen that our features have a low correlation with the yield so the predictive power of these models is logically very low. We'll now try to do binary classification, maybe it could work better. Moreover, we'll focus on more complex models to see if they can extract non linear relationships between features and yields.

# III - Binary classification models

In [6]:
datapath = os.path.join('data', 'US', 'us_data_weekly.csv')
dus_weekly = pd.read_csv(datapath, index_col=0)
dus_weekly = dus_weekly.dropna()

### A) Creating new features

In [7]:

def add_ts_features(df, cols, max_lag=20, windows=[10,20,50]):

    all_features = {}

    for col in cols:
        y = df[col]

        # --- Rolling statistics ---
        for w in windows:
            roll = y.rolling(w)
            all_features[f'{col}_mean_{w}'] = roll.mean()
            all_features[f'{col}_std_{w}'] = roll.std()
            all_features[f'{col}_q25_{w}'] = roll.quantile(0.25)
            all_features[f'{col}_q75_{w}'] = roll.quantile(0.75)
            all_features[f'{col}_q05_{w}'] = roll.quantile(0.05)
            all_features[f'{col}_q90_{w}'] = roll.quantile(0.9)
            all_features[f'{col}_range_{w}'] = roll.max() - roll.min()

            # Z-score et momentum
            all_features[f'{col}_zscore_{w}'] = (y - roll.mean()) / roll.std()
            all_features[f'{col}_momentum_{w}'] = y - y.shift(w)

        # Ratio de moyennes rapides / lentes
        all_features[f'{col}_ratio_{10}_{50}'] = (
            y.rolling(10).mean() / y.rolling(50).mean()
        )

        # Volatilité annualisée approx
        all_features[f'{col}_vol_20'] = y.rolling(20).std() * np.sqrt(52)

        # --- Lags bruts ---
        for lag in [1,2,3,4,5,10,15,20,25,30,40,50]:
            all_features[f'{col}_lag_{lag}'] = y.shift(lag-1)

        # --- Autocorrélations (in-sample) ---
        
        acf_vals = acf(y.dropna(), nlags=max_lag, fft=True)
        for lag in range(1, max_lag + 1):
            all_features[f'{col}_autocorr_{lag}'] = acf_vals[lag]

    # --- Construction finale ---
    features = pd.DataFrame(all_features, index=df.index)
    return features


In [8]:
cols = ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3',
        'DGS5', 'DGS7', 'DGS10', 'DGS20', 'DGS30']

dus_features = add_ts_features(dus_weekly, cols)
dus_features= dus_features.dropna()
dus_weekly = dus_weekly.merge(dus_features, how = 'inner', left_index=True, right_index=True)

In [9]:
for col in cols: 
    dus_weekly[f'Y_{col}'] = (dus_weekly[col]>0).astype(int).shift(-1)


dus_weekly= dus_weekly.dropna()
dus_weekly = dus_weekly.replace([np.inf, -np.inf], np.nan)
dus_weekly = dus_weekly.ffill()

#we can now remove the original yield columns
dus_weekly = dus_weekly.drop(columns=cols)

In [10]:
Yw = dus_weekly[[f'Y_{col}' for col in cols]]
Xw = dus_weekly.drop(columns = [f'Y_{col}' for col in cols])

print(f"there are {Xw.shape[1]} features in the dataset")

there are 738 features in the dataset


We augmented the dataset by adding a very large amount of features created from the yields time series. however, many of these features are not informative so we need to remove them before training our model. 

We'll filter features by keeping only those with a mutual information score above than a given threshold. We'll create several datasets of features containing features filtered for the thresholds 0.03, 0.035, 0.04, 0.05 and train models on these specific datasets. 

In [11]:
initial_number_of_features = Xw.shape[1]

threshold_list = [0.03,0.035,0.04,0.05]

datasets = {t:pd.DataFrame() for t in threshold_list}

for threshold_mi in tqdm(threshold_list): 
    selected_features_w = set()
    for col in Yw.columns:
        mi = mutual_info_classif(Xw, Yw[col])
        top_features_i = Xw.columns[mi > threshold_mi]  
        selected_features_w.update(top_features_i)

    datasets[threshold_mi] = Xw[list(selected_features_w)]

    print(f'we removed {initial_number_of_features-len(selected_features_w)} features in the weekly dataset for threshold {threshold_mi}')
    print(f'Number of variables in the dataset with MI threshold {threshold_mi}:',datasets[threshold_mi].shape[1])
    print(f'Macro and market variables in the dataset with MI threshold {threshold_mi}:', [col for col in datasets[threshold_mi].columns if 'DGS' not in col])
    

 25%|██▌       | 1/4 [00:42<02:07, 42.63s/it]

we removed 464 features in the weekly dataset for threshold 0.03
Number of variables in the dataset with MI threshold 0.03: 274
Macro and market variables in the dataset with MI threshold 0.03: ['UNRATE', 'USGOVT', 'SRVPRD', 'MANEMP', 'USTPU', 'M2SL', 'USCONS', 'PERMIT', 'IRLTLT01FRM156N', 'CURRCIR', 'GFDEGDQ188S', 'DNDGRG3M086SBEA', 'IRLTLT01DEM156N', 'CPITRNSL', 'DTCTHFNM', 'FGDEF', 'IRLTLT01JPM156N', 'IRLTLT01AUM156N']


 50%|█████     | 2/4 [01:34<01:35, 47.81s/it]

we removed 559 features in the weekly dataset for threshold 0.035
Number of variables in the dataset with MI threshold 0.035: 179
Macro and market variables in the dataset with MI threshold 0.035: ['USTPU', 'TOTRESNS', 'UEMP5TO14', 'USCONS', 'IRLTLT01CAM156N', 'IRLTLT01DEM156N', 'IRLTLT01AUM156N']


 75%|███████▌  | 3/4 [02:16<00:45, 45.35s/it]

we removed 639 features in the weekly dataset for threshold 0.04
Number of variables in the dataset with MI threshold 0.04: 99
Macro and market variables in the dataset with MI threshold 0.04: ['SRVPRD', 'MTSDS133FMS', 'USTPU', 'GFDEGDQ188S', 'IRLTLT01DEM156N']


100%|██████████| 4/4 [02:53<00:00, 43.48s/it]

we removed 698 features in the weekly dataset for threshold 0.05
Number of variables in the dataset with MI threshold 0.05: 40
Macro and market variables in the dataset with MI threshold 0.05: ['MTSDS133FMS']


Notice that many of the macro and market variables that we extracted from the FRED website do not share much mutual information with the target since many of these variables are removed with the threshold we used. 


### B) Logistic regression 

We define the following pipelines for our logistic regression models.
We train our model on a rolling window of 15 years of data (with cross validation without leakage of future information) and then predict the 4 next weeks. We then update the rolling window, train another model, predict the 4 next weekds and so on. 

In [12]:
window_train = 52 * 15
window_pred = 4

alphas = np.logspace(-5, 2, 30)
tscv = TimeSeriesSplit(n_splits=4)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', MultiOutputClassifier(
        LogisticRegressionCV(
            penalty='l2',
            cv=tscv,
            Cs=alphas,
            fit_intercept=False,
            scoring='accuracy',
            max_iter=5000)))
])

pipepca = Pipeline([
    ('scaler', StandardScaler()),
    ('pca',PCA(n_components=0.99)), 
    ('logreg', MultiOutputClassifier(
        LogisticRegressionCV(
            penalty='l2',
            cv=tscv,
            Cs=alphas,
            fit_intercept=False,
            scoring='accuracy',
            max_iter=5000)))
])

We train logistic regression models for the 4 datasets containing features with mutual information larger than 0.03, 0.035, 0.04, 0.05 respectively. Then we save the results in datasets. 

The dataset with variables with mutual information > 0.03 contains 266 features, which is very high compared to the numer of points in the training datasets (52*15 lines), so we'll also train a model that performs a PCA before applying the logistic regression in order to reduce the number of features and see if it can improve the performance.

In [ ]:
for threshold in threshold_list:

    print(f'training model on dataset with features with mutual information above {threshold}')

    Xw = datasets[threshold]

    # for the dataset containing features with mutual information above 0.03:
    # we'll train our logistic model with and without PCA since there are many features in this dataset
    if threshold == 0.03:


        accuraciesl2 = {col: [] for col in Yw.columns}
        accuraciesl2pca = {col: [] for col in Yw.columns}
        alphal2 = {col: [] for col in Yw.columns}
        alphal2pca = {col: [] for col in Yw.columns}
        feature_importancel2 = {col: [] for col in Yw.columns}
        feature_importancel2pca = {col: [] for col in Yw.columns}
        y_true = []
        y_predl2 = []
        y_predl2pca = []



        for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

        
            pipe.fit(Xw_train, Yw_train)
            pipepca.fit(Xw_train, Yw_train)


            Yw_pred = pipe.predict(Xw_test)
            Yw_predpca = pipepca.predict(Xw_test)

            y_predl2.append(Yw_pred)
            y_predl2pca.append(Yw_predpca)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuraciesl2[col].append(acc)

                acc = accuracy_score(Yw_test[col], Yw_predpca[:, i])
                accuraciesl2pca[col].append(acc)

                # parameters chosen
                best_C = pipe.named_steps['logreg'].estimators_[i].C_[0]
                alphal2[col].append(best_C)

                best_C = pipepca.named_steps['logreg'].estimators_[i].C_[0]
                alphal2pca[col].append(best_C)

                # feature importance
                coefs = np.abs(pipe.named_steps['logreg'].estimators_[i].coef_).flatten()
                feature_importancel2[col].append(coefs)

                coefs = np.abs(pipepca.named_steps['logreg'].estimators_[i].coef_).flatten()
                feature_importancel2pca[col].append(coefs)

        ### saving the results
        acc = pd.DataFrame(accuraciesl2,columns = Yw.columns)
        params = pd.DataFrame(alphal2, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importancel2, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_predl2).reshape(-1, np.array(y_predl2).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)


        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} is:')
        print(acc.mean())
        acc.to_csv('results of models/accuracies logistic regression, 15y train test.csv')
        params.to_csv('results of models/params logistic regression, 15y train test.csv')
        feature_imp.to_csv('results of models/feature importance logistic regression, 15y train test.csv')
        ypred.to_csv('results of models/forecast logistic regression, 15y train test.csv')
        ytrue.to_csv('results of models/true values logistic regression, 15y train test.csv')

        acc = pd.DataFrame(accuraciesl2pca,columns = Yw.columns)
        params = pd.DataFrame(alphal2pca, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importancel2pca, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_predl2pca).reshape(-1, np.array(y_predl2pca).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} and PCA is:')
        print(acc.mean())
        acc.to_csv('results of models/accuracies logistic regression pca, 15y train test.csv')
        params.to_csv('results of models/params logistic regression pca, 15y train test.csv')
        feature_imp.to_csv('results of models/feature importance logistic regression pca, 15y train test.csv')
        ypred.to_csv('results of models/forecast logistic regression pca, 15y train test.csv')
        ytrue.to_csv('results of models/true values logistic regression pca, 15y train test.csv')

    #for other datasets, we train models without applying a PCA before. 
    else: 


        accuraciesl2 = {col: [] for col in Yw.columns}
        alphal2 = {col: [] for col in Yw.columns}
        feature_importancel2 = {col: [] for col in Yw.columns}
        y_true = []
        y_predl2 = []

        for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):

            end_train = start + window_train
            end_pred = end_train + window_pred
            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]
            y_true.append(Yw_test)

            pipe.fit(Xw_train, Yw_train)

            Yw_pred = pipe.predict(Xw_test)

            y_predl2.append(Yw_pred)


            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuraciesl2[col].append(acc)


                # alpha optimal choisi
                best_C = pipe.named_steps['logreg'].estimators_[i].C_[0]
                alphal2[col].append(best_C)

                # importance des features = moyenne absolue des coefficients
                coefs = np.abs(pipe.named_steps['logreg'].estimators_[i].coef_).flatten()
                feature_importancel2[col].append(coefs)

        acc = pd.DataFrame(accuraciesl2,columns = Yw.columns)
        params = pd.DataFrame(alphal2, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importancel2, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_predl2).reshape(-1, np.array(y_predl2).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} is:')
        print(acc.mean())
        acc.to_csv(f'results of models/accuracies logistic regression, MI {threshold}, 15y train test.csv')
        params.to_csv(f'results of models/params logistic regression, MI {threshold}, 15y train test.csv')
        feature_imp.to_csv(f'results of models/feature importance logistic regression, MI {threshold}, 15y train test.csv')
        ypred.to_csv(f'results of models/forecast logistic regression, MI {threshold}, 15y train test.csv')
        ytrue.to_csv(f'results of models/true values logistic regression, MI {threshold}, 15y train test.csv')


training model on dataset with features with mutual information above 0.03


100%|██████████| 79/79 [35:56<00:00, 27.29s/it]


For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > 0.03 is:
Y_DGS1MO    0.607595
Y_DGS3MO    0.661392
Y_DGS6MO    0.636076
Y_DGS1      0.591772
Y_DGS2      0.582278
Y_DGS3      0.563291
Y_DGS5      0.525316
Y_DGS7      0.496835
Y_DGS10     0.487342
Y_DGS20     0.509494
Y_DGS30     0.515823
dtype: float64
For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > 0.03 and PCA is:
Y_DGS1MO    0.620253
Y_DGS3MO    0.667722
Y_DGS6MO    0.651899
Y_DGS1      0.588608
Y_DGS2      0.579114
Y_DGS3      0.537975
Y_DGS5      0.534810
Y_DGS7      0.537975
Y_DGS10     0.496835
Y_DGS20     0.512658
Y_DGS30     0.490506
dtype: float64
training model on dataset with features with mutual information above 0.035


100%|██████████| 79/79 [24:58<00:00, 18.96s/it]


For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > 0.035 is:
Y_DGS1MO    0.613924
Y_DGS3MO    0.702532
Y_DGS6MO    0.645570
Y_DGS1      0.544304
Y_DGS2      0.515823
Y_DGS3      0.556962
Y_DGS5      0.509494
Y_DGS7      0.474684
Y_DGS10     0.443038
Y_DGS20     0.487342
Y_DGS30     0.465190
dtype: float64
training model on dataset with features with mutual information above 0.04


100%|██████████| 79/79 [17:00<00:00, 12.92s/it]


For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > 0.04 is:
Y_DGS1MO    0.636076
Y_DGS3MO    0.702532
Y_DGS6MO    0.686709
Y_DGS1      0.598101
Y_DGS2      0.525316
Y_DGS3      0.518987
Y_DGS5      0.525316
Y_DGS7      0.477848
Y_DGS10     0.493671
Y_DGS20     0.471519
Y_DGS30     0.474684
dtype: float64
training model on dataset with features with mutual information above 0.05


100%|██████████| 79/79 [07:56<00:00,  6.03s/it]


For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > 0.05 is:
Y_DGS1MO    0.645570
Y_DGS3MO    0.686709
Y_DGS6MO    0.629747
Y_DGS1      0.598101
Y_DGS2      0.550633
Y_DGS3      0.531646
Y_DGS5      0.512658
Y_DGS7      0.471519
Y_DGS10     0.484177
Y_DGS20     0.458861
Y_DGS30     0.471519
dtype: float64


## C) XGBoost

We'll now train XGboost models to see whether it can capture non linearities in the data and improve accuracy of out-of-sample predictions. Once again, we'll train some models on the 4 datasets that we created. 

For the first dataset (with features with a MI>0.03), we'll also train a model that performs a PCA to reduce the number of features before applying XGboost. 

The pipelines that we'll use are the following:

In [ ]:
window_train = 52 * 15
window_pred = 4
tscv = TimeSeriesSplit(n_splits=4)


# parameters to test in cross validation 
param_grid = {
 'xgb__estimator__n_estimators': [100, 200],
 'xgb__estimator__learning_rate': [0.01, 0.05],
 'xgb__estimator__max_depth': [4,7],
 'xgb__estimator__subsample': [0.5, 0.7],
 'xgb__estimator__colsample_bytree': [0.4, 0.8],
 'xgb__estimator__min_child_weight': [5, 10],
 'xgb__estimator__reg_alpha': [0.5, 1.0],
 'xgb__estimator__reg_lambda': [5, 10]
}




#pipeline without PCA 
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', MultiOutputClassifier(
        XGBClassifier(
            objective='binary:logistic',
            use_label_encoder=False,
            n_jobs=-1,
            random_state=42,
            verbosity=0
        )
    ))
])


#pipeline with PCA 
pipepca = Pipeline([
    ('scaler', StandardScaler()),
    ('pca',PCA(n_components = 0.99)),
    ('xgb', MultiOutputClassifier(
        XGBClassifier(
            objective='binary:logistic',
            use_label_encoder=False,
            n_jobs=-1,
            random_state=42,
            verbosity=0
        )
    ))
])


#and we'll use gridsearchCV to cross validate the model:
GridSearchCV(pipepca, #or pipe
            param_grid,
            cv=tscv,
            scoring='accuracy',
            n_jobs=-1,
            verbose=0)


We now train XGBoost models on the datasets:

In [ ]:
for threshold in threshold_list:

    print(f'training model on dataset with features with mutual information above {threshold}')

    Xw = datasets[threshold]

    # for the dataset containing features with mutual information above 0.03:
    # we'll train our xgboost model with and without PCA since there are many features in this dataset
    if threshold == 0.03:

        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []

        for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(pipe,
                                param_grid,
                                cv=tscv,
                                scoring='accuracy',
                                n_jobs=-1,
                                verbose=0)

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)
                print(col, np.mean(accuracies[col]))

                # feature importance pour la i-ème sortie
                est = best_model.named_steps['xgb'].estimators_[i]   # XGBClassifier
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)



        acc = pd.DataFrame(accuracies,columns = Yw.columns)
        params = pd.DataFrame(params_by_target, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} is:')
        print(acc.mean())
        acc.to_csv('results of models/accuracies xgboost, 15y train test.csv')
        params.to_csv('results of models/params xgboost, 15y train test.csv')
        feature_imp.to_csv('results of models/feature importance xgboost, 15y train test.csv')
        ypred.to_csv('results of models/forecast xgboost, 15y train test.csv')
        ytrue.to_csv('results of models/true values xgboost, 15y train test.csv')





        #### Now we do the same training but this time we add the PCA in the pipeline before training the model
        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []


        
        for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(pipepca,
                                param_grid,
                                cv=tscv,
                                scoring='accuracy',
                                n_jobs=-1,
                                verbose=0)

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)
                print(col, np.mean(accuracies[col]))

                # feature importance pour la i-ème sortie
                est = best_model.named_steps['xgb'].estimators_[i]   # XGBClassifier
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)



        acc = pd.DataFrame(accuracies,columns = Yw.columns)
        params = pd.DataFrame(params_by_target, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} and PCA is:')
        print(acc.mean())
        acc.to_csv('results of models/accuracies xgboost, pca, 15y train test.csv')
        params.to_csv('results of models/params xgboost, pca, 15y train test.csv')
        feature_imp.to_csv('results of models/feature importance xgboost, pca, 15y train test.csv')
        ypred.to_csv('results of models/forecast xgboost, pca, 15y train test.csv')
        ytrue.to_csv('results of models/true values xgboost, pca, 15y train test.csv')
    

    # for other datasets with larger mutual information threshold, we don't apply PCA and train the model directly 
    else:
        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []

        for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(pipe,
                                param_grid,
                                cv=tscv,
                                scoring='accuracy',
                                n_jobs=-1,
                                verbose=0)

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)
                print(col, np.mean(accuracies[col]))

                # feature importance pour la i-ème sortie
                est = best_model.named_steps['xgb'].estimators_[i]   # XGBClassifier
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)


        
        acc = pd.DataFrame(accuracies,columns = Yw.columns)
        params = pd.DataFrame(params_by_target, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} is:')
        print(acc.mean())
        acc.to_csv(f'results of models/accuracies xgboost, MI {threshold}, 15y train test.csv')
        params.to_csv(f'results of models/params xgboost, MI {threshold}, 15y train test.csv')
        feature_imp.to_csv(f'results of models/feature importance xgboost, MI {threshold}, 15y train test.csv')
        ypred.to_csv(f'results of models/forecast xgboost, MI {threshold}, 15y train test.csv')
        ytrue.to_csv(f'results of models/true values xgboost, MI {threshold}, 15y train test.csv')


## D) Random Forests

We'll proceed as for the logistic regression and XGBoost models when training random forests. The pipelines (with and without PCA) are defined below:

In [15]:
window_train = 52 * 15
window_pred = 4
tscv = TimeSeriesSplit(n_splits=4)



param_grid = {
    'rf__estimator__n_estimators': [150],
    'rf__estimator__max_depth': [4,7],
    'rf__estimator__min_samples_leaf': [5,10]
}

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', MultiOutputClassifier(
        RandomForestClassifier(
            n_jobs=-1,
            random_state = 42
            )))
])

pipepca = Pipeline([
    ('scaler', StandardScaler()),
    ('pca',PCA(n_components=0.99)),
    ('rf', MultiOutputClassifier(
        RandomForestClassifier(
            n_jobs=-1,
            random_state = 42
            )))
])


#we will cross validate the model the following way: 
grid = GridSearchCV(pipe,  #or pipepca
                        param_grid,
                        cv = tscv,
                        scoring = 'accuracy',
                        n_jobs=-1
                        )

In [ ]:
for threshold in threshold_list:

    print(f'training model on dataset with features with mutual information above {threshold}')

    Xw = datasets[threshold]

    # for the dataset containing features with mutual information above 0.03:
    # we'll train our random forest model with and without PCA since there are many features in this dataset
    if threshold == 0.03:

        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []

        for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(pipe,
                                param_grid,
                                cv=tscv,
                                scoring='accuracy',
                                n_jobs=-1,
                                verbose=0)

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)
           
                # feature importance pour la i-ème sortie
                est = best_model.named_steps['rf'].estimators_[i]  
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)



        acc = pd.DataFrame(accuracies,columns = Yw.columns)
        params = pd.DataFrame(params_by_target, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} is:')
        print(acc.mean())
        acc.to_csv('results of models/accuracies random forest, 15y train test.csv')
        params.to_csv('results of models/params random forest, 15y train test.csv')
        feature_imp.to_csv('results of models/feature importance random forest, 15y train test.csv')
        ypred.to_csv('results of models/forecast random forest, 15y train test.csv')
        ytrue.to_csv('results of models/true values random forest, 15y train test.csv')





        #### Now we do the same training but this time we add the PCA in the pipeline before training the model
        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []


        
        for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(pipepca,
                                param_grid,
                                cv=tscv,
                                scoring='accuracy',
                                n_jobs=-1,
                                verbose=0)

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)

                # feature importance pour la i-ème sortie
                est = best_model.named_steps['rf'].estimators_[i]   
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)



        acc = pd.DataFrame(accuracies,columns = Yw.columns)
        params = pd.DataFrame(params_by_target, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} and PCA is:')
        print(acc.mean())
        acc.to_csv('results of models/accuracies random forest, pca, 15y train test.csv')
        params.to_csv('results of models/params random forest, pca, 15y train test.csv')
        feature_imp.to_csv('results of models/feature importance random forest, pca, 15y train test.csv')
        ypred.to_csv('results of models/forecast random forest, pca, 15y train test.csv')
        ytrue.to_csv('results of models/true values random forest, pca, 15y train test.csv')
    

    # for other datasets with larger mutual information threshold, we don't apply PCA and train the model directly 
    else:
        accuracies = {col: [] for col in Yw.columns}
        params_by_target = {col: [] for col in Yw.columns}
        feature_importance = {col: [] for col in Yw.columns}
        y_true = []
        y_pred = []

        for start in tqdm(range(0, len(Xw) - window_train - window_pred + 1, window_pred)):
            end_train = start + window_train
            end_pred = end_train + window_pred

            Xw_train = Xw.iloc[start:end_train]
            Yw_train = Yw.iloc[start:end_train]
            Xw_test = Xw.iloc[end_train:end_pred]
            Yw_test = Yw.iloc[end_train:end_pred]

            y_true.append(Yw_test)

            grid = GridSearchCV(pipe,
                                param_grid,
                                cv=tscv,
                                scoring='accuracy',
                                n_jobs=-1,
                                verbose=0)

            grid.fit(Xw_train, Yw_train)
            best_model = grid.best_estimator_

            Yw_pred = best_model.predict(Xw_test)
            y_pred.append(Yw_pred)

            for i, col in enumerate(Yw_train.columns):
                acc = accuracy_score(Yw_test[col], Yw_pred[:, i])
                accuracies[col].append(acc)
            

                # feature importance pour la i-ème sortie
                est = best_model.named_steps['rf'].estimators_[i]  
                importance = est.feature_importances_
                feature_importance[col].append(importance)

                # hyperparamètres de l'estimateur i
                params = est.get_params()
                params_by_target[col].append(params)


        
        acc = pd.DataFrame(accuracies,columns = Yw.columns)
        params = pd.DataFrame(params_by_target, columns = Yw.columns) 
        feature_imp = pd.DataFrame(feature_importance, columns = Yw.columns)
        Y_true_flat = np.array(y_true).reshape(-1, np.array(y_true).shape[2])  # aplati les 2 premières dimensions en une seule
        ytrue = pd.DataFrame(Y_true_flat, index = Yw.iloc[780:Yw.shape[0]-1].index,columns=Yw.columns)
        Y_pred_flat = np.array(y_pred).reshape(-1, np.array(y_pred).shape[2])  # aplati les 2 premières dimensions en une seule
        ypred = pd.DataFrame(Y_pred_flat,index = Yw.iloc[780:Yw.shape[0]-1].index, columns=Yw.columns)

        print(f'For each yield, the average out-of-sample accuracy over the 79 rolling windows on the dataset with mutual information > {threshold} is:')
        print(acc.mean())
        acc.to_csv(f'results of models/accuracies random forest, MI {threshold}, 15y train test.csv')
        params.to_csv(f'results of models/params random forest, MI {threshold}, 15y train test.csv')
        feature_imp.to_csv(f'results of models/feature importance random forest, MI {threshold}, 15y train test.csv')
        ypred.to_csv(f'results of models/forecast random forest, MI {threshold}, 15y train test.csv')
        ytrue.to_csv(f'results of models/true values random forest, MI {threshold}, 15y train test.csv')


# IV - Models analysis 

Now, we can observe and analyze the average out-of-sample accuracy of all the models that were trained over the 79 fifteen years rolling windows:

In [ ]:
accuracies_lr = {
            "lr 0.03":pd.read_csv("results of models/accuracies logistic regression, 15y train test.csv",index_col=0).mean(),
             "lr 0.03 pca":pd.read_csv("results of models/accuracies logistic regression pca, 15y train test.csv", index_col=0).mean(),
             "lr 0.035":pd.read_csv("results of models/accuracies logistic regression, MI 0.035, 15y train test.csv", index_col=0).mean(),
             "lr 0.04":pd.read_csv("results of models/accuracies logistic regression, MI 0.04, 15y train test.csv", index_col=0).mean(),
             "lr 0.05":pd.read_csv("results of models/accuracies logistic regression, MI 0.05, 15y train test.csv", index_col=0).mean()
             }
accuracies_lr = pd.DataFrame(accuracies_lr)

accuracies_rf = {
            "rf 0.03":pd.read_csv("results of models/accuracies random forest, no pca, 15y train test.csv", index_col=0).mean(),
             "rf 0.03 pca":pd.read_csv("results of models/accuracies random forest pca, 15y train test.csv", index_col=0).mean(),
             "rf 0.035":pd.read_csv("results of models/accuracies random forest, MI 0.035, 15y train test.csv", index_col=0).mean(),
             "rf 0.04":pd.read_csv("results of models/accuracies random forest, MI 0.04, 15y train test.csv", index_col=0).mean(),
             "rf 0.05":pd.read_csv("results of models/accuracies random forest, MI 0.05, 15y train test.csv", index_col=0).mean()
             }
accuracies_rf = pd.DataFrame(accuracies_rf)

accuracies_xg = {
    ################ ENLEVER LES # QUAND AUDRIC AURA PUSH LES TRUCS ###########
           # "xg 0.03":pd.read_csv("results of models/accuracies xgboost, no pca, 15y train test.csv", index_col=0).mean(),
             "xg 0.03 pca":pd.read_csv("results of models/accuracies xgboost, pca, 15y train test.csv", index_col=0).mean(),
           #  "xg 0.035":pd.read_csv("results of models/accuracies xgboost, MI 0.035, 15y train test.csv").mean(),
             "xg 0.04":pd.read_csv("results of models/accuracies xgboost, MI 0.04, 15y train test.csv", index_col=0).mean(),
             "xg 0.05":pd.read_csv("results of models/accuracies xgboost, MI 0.05, 15y train test.csv", index_col=0).mean()
             }
accuracies_xg = pd.DataFrame(accuracies_xg)


# AUDRIC PEUX TU STP FAIRE LA MEME AVEC TES DATASETS LSTM 
#accuracies_lstm ={}

In [37]:
accuracies = pd.concat([accuracies_lr,accuracies_rf,accuracies_xg],axis = 1).round(2)
accuracies.index = ["1m yield","3m yield","6m yield","1y yield","2y yield","3y yield","5y yield","7y yield","10y yield","20y yield","30y yield"]
accuracies

,lr 0.03,lr 0.03 pca,lr 0.035,lr 0.04,lr 0.05,rf 0.03,rf 0.03 pca,rf 0.035,rf 0.04,rf 0.05,xg 0.03 pca,xg 0.04,xg 0.05
1m yield,0.58,0.61,0.62,0.64,0.63,0.64,0.61,0.64,0.64,0.65,0.59,0.61,0.62
3m yield,0.68,0.67,0.71,0.71,0.65,0.70,0.62,0.70,0.70,0.69,0.55,0.69,0.68
6m yield,0.64,0.63,0.68,0.65,0.64,0.67,0.57,0.67,0.67,0.67,0.50,0.67,0.66
1y yield,0.57,0.57,0.62,0.62,0.59,0.59,0.54,0.59,0.59,0.60,0.56,0.52,0.56
2y yield,0.56,0.59,0.58,0.54,0.55,0.56,0.54,0.56,0.53,0.55,0.53,0.52,0.53
3y yield,0.55,0.55,0.53,0.54,0.53,0.53,0.53,0.53,0.55,0.56,0.51,0.49,0.50
5y yield,0.50,0.55,0.55,0.54,0.50,0.54,0.56,0.51,0.52,0.52,0.51,0.50,0.49
7y yield,0.55,0.53,0.53,0.53,0.51,0.54,0.54,0.51,0.52,0.55,0.50,0.50,0.51
10y yield,0.54,0.52,0.48,0.48,0.48,0.51,0.52,0.49,0.50,0.53,0.51,0.47,0.50
20y yield,0.50,0.51,0.51,0.49,0.43,0.49,0.53,0.47,0.49,0.53,0.52,0.50,0.51


For each maturity in the yield curve, we will identify the model providing the best average out-of-sample accuracy over the 79 15 years rolling training periods:

In [ ]:
best_models = pd.DataFrame({"Best model":accuracies.idxmax(axis=1),
                            "Best average out-of-sample accuracy": accuracies.max(axis=1)
                            })
best_models 

,Best model,Best average out-of-sample accuracy
1m yield,rf 0.05,0.65
3m yield,lr 0.035,0.71
6m yield,lr 0.035,0.68
1y yield,lr 0.035,0.62
2y yield,lr 0.03 pca,0.59
3y yield,rf 0.05,0.56
5y yield,rf 0.03 pca,0.56
7y yield,lr 0.03,0.55
10y yield,lr 0.03,0.54
20y yield,rf 0.03 pca,0.53


##### Some important observations on the models, the features used and the overall accuracy:

- we can see that in most cases, logistic regression is the model performing the highest average out-of-sample accuracy. 7 out of 11 yield maturities are better forecast with logistic regression.

- Concerning the 4 other yield maturities that are not better forecast with logistic regression, the model with the highest average out-of-sample accuracy is the random forest.

- Overall, our models manage to get a satisfying out-of-sample accuracy on the short-end of the yield curve, i.e. on yields of bonds with maturity in less than 2 years: for maturities of one year or less, we manage to get average out-of-sample accuracies above 60% and even up to 71%. We are rather satisfied with these figures and these models could be useful signals to implement trading strategies. However we could not implement a strategy as we could not access data on US bond prices.

- It is more difficult to forecast the medium-end and long-end of the yield curve (yields of bonds with maturity above 1y): for these yields, we get an average out-of-sample accuracy between 0.53 and 0.59. This is less satisfying and our models provide a weaker signal on those yields. The lower predictive power of our models on the long-end of the yield curve is not surprising, because long-term yields are not as much driven by macroeconomic, financial and monetary variables as short-term yields. Long-term yields mostly depend on expectations of future short rates over many years, which are latent and difficult to model and observe.

- Interestingly, the random forest sometimes performs better than logistic regression to forecast the long-end of the yield curve. This could suggest that there is some non-linear relationship between some features and the long-term yields that is not captured very well by a linear model like logistic regression.

- Even if in some cases, random forests provide a higher out-of-sample accuracy than linear regression, this improvement is only marginal: for instance, to forecast the 5y yield, logistic regression has a 55% out-of-sample accuracy and random forest 56%. This improvement may be marginal compared to the increase in complexity and the loss of interpretability of the model.

- For most yields, the datasets containing features with mutual information above 0.03 or 0.035 are the datasets providing the best trained models. These are the datasets containing a set of macroeconomic and financial variables, as well as functions of the past yields (lags, rolling means, quantiles, ...).

- However, when it comes to the shortest maturity yield, ie the 1 month yield, we see that the dataset providing the trained model with the best out-of-sample accuracy is the dataset containing features with mutual information above 0.05. This dataset contains only functions of the past yield, so it is interesting to see that we can obtain a 65% out-of-sample accuracy to predict 1 month yields only with previous values of the yields. This could suggest that the trend-following and mean-reverting behavior of the 1 month yield is stronger than for other yields, and that it could be suited for trading strategies exploiting this behavior.


We'll now perform a Diebold Mariano test to ensure that our forecast performance is statistically significant. 

In [44]:
forecast_lr = {
            "lr 0.03":pd.read_csv("results of models/forecast logistic regression, 15y train test.csv",index_col=0),
             "lr 0.03 pca":pd.read_csv("results of models/forecast logistic regression pca, 15y train test.csv", index_col=0),
             "lr 0.035":pd.read_csv("results of models/forecast logistic regression, MI 0.035, 15y train test.csv", index_col=0),
             "lr 0.04":pd.read_csv("results of models/forecast logistic regression, MI 0.04, 15y train test.csv", index_col=0),
             "lr 0.05":pd.read_csv("results of models/forecast logistic regression, MI 0.05, 15y train test.csv", index_col=0)
             }
forecast_lr = pd.DataFrame(forecast_lr)

forecast_rf = {
            "rf 0.03":pd.read_csv("results of models/forecast random forest, no pca, 15y train test.csv", index_col=0),
             "rf 0.03 pca":pd.read_csv("results of models/forecast random forest pca, 15y train test.csv", index_col=0),
             "rf 0.035":pd.read_csv("results of models/forecast random forest, MI 0.035, 15y train test.csv", index_col=0),
             "rf 0.04":pd.read_csv("results of models/forecast random forest, MI 0.04, 15y train test.csv", index_col=0),
             "rf 0.05":pd.read_csv("results of models/forecast random forest, MI 0.05, 15y train test.csv", index_col=0)
             }
forecast_rf = pd.DataFrame(forecast_rf)

forecast_xg = {
    ################ ENLEVER LES # QUAND AUDRIC AURA PUSH LES TRUCS ###########
           # "xg 0.03":pd.read_csv("results of models/forecast xgboost, no pca, 15y train test.csv", index_col=0),
             "xg 0.03 pca":pd.read_csv("results of models/forecast xgboost, pca, 15y train test.csv", index_col=0),
           #  "xg 0.035":pd.read_csv("results of models/forecast xgboost, MI 0.035, 15y train test.csv"),
             "xg 0.04":pd.read_csv("results of models/forecast xgboost, MI 0.04, 15y train test.csv", index_col=0),
             "xg 0.05":pd.read_csv("results of models/forecast xgboost, MI 0.05, 15y train test.csv", index_col=0)
             }
forecast_xg = pd.DataFrame(forecast_xg)


# AUDRIC PEUX TU STP FAIRE LA MEME AVEC TES DATASETS LSTM 
#forecast_lstm ={}

ValueError: If using all scalar values, you must pass an index

In [45]:
forecast_lr

{'lr 0.03':             Y_DGS1MO  Y_DGS3MO  Y_DGS6MO  Y_DGS1  Y_DGS2  Y_DGS3  Y_DGS5  \
 2018-12-07       1.0       1.0       1.0     1.0     0.0     1.0     0.0   
 2018-12-14       1.0       1.0       1.0     1.0     1.0     1.0     1.0   
 2018-12-21       1.0       1.0       1.0     1.0     1.0     1.0     1.0   
 2018-12-28       1.0       1.0       1.0     1.0     1.0     1.0     1.0   
 2019-01-04       1.0       1.0       1.0     1.0     1.0     1.0     1.0   
 ...              ...       ...       ...     ...     ...     ...     ...   
 2024-11-22       0.0       0.0       0.0     0.0     0.0     1.0     1.0   
 2024-11-29       0.0       0.0       1.0     1.0     1.0     1.0     0.0   
 2024-12-06       1.0       0.0       0.0     0.0     1.0     1.0     0.0   
 2024-12-13       1.0       0.0       0.0     0.0     0.0     0.0     0.0   
 2024-12-20       1.0       0.0       0.0     0.0     0.0     0.0     0.0   
 
             Y_DGS7  Y_DGS10  Y_DGS20  Y_DGS30  
 2018-12-07   